In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
data = pd.read_csv('Intractable_Bayesian_Predictions.csv')
data.head()

,Unnamed: 0,Ticker,Hv10,IvMean10,IVTarget,HV_Target,Gamma(Uniform_Exponential)_Mean,Gamma(Uniform_Exponential)_Median,Gamma(Uniform_Exponential)_Lower,Gamma(Uniform_Exponential)_Upper,...,Gamma(Uniform_Uniform)_Lower,Gamma(Uniform_Uniform)_Upper,Gamma(Exponential_Exponential)_Mean,Gamma(Exponential_Exponential)_Median,Gamma(Exponential_Exponential)_Lower,Gamma(Exponential_Exponential)_Upper,LogNormal(Uniform_Exponential)_Mean,LogNormal(Uniform_Exponential)_Median,LogNormal(Uniform_Exponential)_Lower,LogNormal(Uniform_Exponential)_Upper
0,0,LUMN,0.1799,0.2706,0.1834,0.2454,0.309664,0.207567,0.045723,0.592191,...,0.048578,1.395590,0.307580,0.271438,0.132127,0.486504,3.163117,1.033775,0.223677,4.755219
1,1,LUMN,0.1785,0.2926,0.2017,0.3096,0.369202,0.247919,0.055027,0.708198,...,0.065936,1.501334,0.372167,0.336432,0.178262,0.570609,2.231625,1.026795,0.289602,3.641099
2,2,LUMN,0.4154,0.4265,0.3620,0.4792,0.442485,0.294056,0.065475,0.845816,...,0.074837,1.552043,0.444660,0.377473,0.160801,0.739489,2.263932,1.023273,0.286073,3.688284
3,3,LUMN,0.3697,0.3851,0.3511,0.3480,0.375391,0.253859,0.057272,0.710047,...,0.071489,1.534015,0.373240,0.343894,0.194680,0.555547,2.093944,1.021314,0.299985,3.465987
4,4,LUMN,0.2062,0.4686,0.2842,0.4523,0.556329,0.374614,0.082400,1.063385,...,0.109546,1.741721,0.549643,0.488713,0.241948,0.863676,1.549426,1.012931,0.396197,2.600837


In [3]:
def add_sectors(dataframe):
    sectors = pd.read_csv('sectors.csv')
    final = dataframe.merge(sectors, on='Ticker',how='inner')
    return(final)

def split_sectors(data,sectors):
    num_splits = len(sectors)
    dataframes = []
    
    for i in range(0,num_splits):
        dataframes.append(data[data['Sector'] == sectors[i]])
        
    final = pd.concat(dataframes,axis = 0)
    return(final)


High_vol_sectors = ['Information Technology','Consumer Discretionary','Health Care','Energy','Communication Services']
Medium_vol_sectors = ['Materials','Industrials','Financials',]
Low_vol_sectors = ['Utilities','Real Estate','Consumer Staples']

In [4]:
def condense_results(dataframe, name ,sectors = ['all']):
    
    if sectors[0] != 'all':
        
        dataframe = add_sectors(dataframe)
        dataframe = split_sectors(dataframe,sectors)
        dataframe = dataframe.iloc[:,[0,1,2,3,4,5,6,7]]
                    
        
    result = []
        
    result.append(name)
    Confidence_Interval = 0
    Mean_Median_Interval = 0
    Mean_Direction = 0
    Median_Direction = 0
    mean_pred = []
    median_pred = []
    target = []

    for row in dataframe.itertuples():
            
        mean_pred.append(row[2])
        median_pred.append(row[3])
        target.append(row[8])
            
        if row[7] > row[5] and row[7] < row[6]:
            Confidence_Interval += 1
                
        if row[7] > (row[3]-row[4]) and row[7] < (row[2]+row[4]):
                
            Mean_Median_Interval += 1
                
        if row[2] > row[8] and row[7] > row[8]:
            Mean_Direction += 1
            
        elif row[2] < row[8] and row[7] < row[8]:
            Mean_Direction += 1
                
        if row[3] > row[8] and row[7] > row[8]:
            Median_Direction += 1
            
        elif row[3] < row[8] and row[7] < row[8]:
            Median_Direction += 1
            
                     
            
    result.append(Confidence_Interval / len(dataframe.index))
    result.append(Mean_Median_Interval / len(dataframe.index))
    result.append(Mean_Direction / len(dataframe.index))
    result.append(Median_Direction / len(dataframe.index))
    mean_MSE = np.square(np.subtract(target,mean_pred)).mean()
    median_MSE = np.square(np.subtract(target,median_pred)).mean()
    mean_rmse = math.sqrt(mean_MSE)
    median_rmse = math.sqrt(median_MSE)
    result.append(mean_rmse)
    result.append(median_rmse)
        
        
    return(result)
                   


In [5]:
def split_dataframes_HV(dataframe):
    
    seperate = []
    
    one = dataframe[['Ticker', 'Gamma(Uniform_Exponential)_Mean','Gamma(Uniform_Exponential)_Median', 'Krim_Distance_one','Gamma(Uniform_Exponential)_Lower', 'Gamma(Uniform_Exponential)_Upper','HV_Target', 'Hv10']]
    two = dataframe[['Ticker', 'Gamma(Uniform_Uniform)_Mean','Gamma(Uniform_Uniform)_Median', 'Krim_Distance_two','Gamma(Uniform_Uniform)_Lower', 'Gamma(Uniform_Uniform)_Upper','HV_Target', 'Hv10']]
    three = dataframe[['Ticker', 'Gamma(Exponential_Exponential)_Mean','Gamma(Exponential_Exponential)_Median', 'Krim_Distance_three','Gamma(Exponential_Exponential)_Lower', 'Gamma(Exponential_Exponential)_Upper','HV_Target', 'Hv10']]
    four = dataframe[['Ticker', 'LogNormal(Uniform_Exponential)_Mean','LogNormal(Uniform_Exponential)_Median', 'Krim_Distance_four','LogNormal(Uniform_Exponential)_Lower', 'LogNormal(Uniform_Exponential)_Upper','HV_Target', 'Hv10']]
    
    seperate.append(one)
    seperate.append(two)
    seperate.append(three)
    seperate.append(four)
    return(seperate)
    

In [6]:
def split_dataframes_IV(dataframe):
    
    seperate = []
    
    one = dataframe[['Ticker', 'Gamma(Uniform_Exponential)_Mean','Gamma(Uniform_Exponential)_Median', 'Krim_Distance_one','Gamma(Uniform_Exponential)_Lower', 'Gamma(Uniform_Exponential)_Upper','IVTarget', 'IvMean10']]
    two = dataframe[['Ticker', 'Gamma(Uniform_Uniform)_Mean','Gamma(Uniform_Uniform)_Median', 'Krim_Distance_two','Gamma(Uniform_Uniform)_Lower', 'Gamma(Uniform_Uniform)_Upper','IVTarget', 'IvMean10']]
    three = dataframe[['Ticker', 'Gamma(Exponential_Exponential)_Mean','Gamma(Exponential_Exponential)_Median', 'Krim_Distance_three','Gamma(Exponential_Exponential)_Lower', 'Gamma(Exponential_Exponential)_Upper','IVTarget', 'IvMean10']]
    four = dataframe[['Ticker', 'LogNormal(Uniform_Exponential)_Mean','LogNormal(Uniform_Exponential)_Median', 'Krim_Distance_four','LogNormal(Uniform_Exponential)_Lower', 'LogNormal(Uniform_Exponential)_Upper','IVTarget', 'IvMean10']]
    
    seperate.append(one)
    seperate.append(two)
    seperate.append(three)
    seperate.append(four)
    return(seperate)

In [7]:
data['Krim_Distance_one'] = data['Gamma(Uniform_Exponential)_Mean'] - data['Gamma(Uniform_Exponential)_Median']
data['Krim_Distance_two'] = data['Gamma(Uniform_Uniform)_Mean'] - data['Gamma(Uniform_Uniform)_Median']
data['Krim_Distance_three'] = data['Gamma(Exponential_Exponential)_Mean'] - data['Gamma(Exponential_Exponential)_Median']
data['Krim_Distance_four'] = data['LogNormal(Uniform_Exponential)_Mean'] - data['LogNormal(Uniform_Exponential)_Median']

# The next four cells are for the HV predictions

In [8]:
HV_Results = pd.DataFrame(columns = ['Model','Credible_Interval','Mean_Median_Interval','Mean_Direction_Acc','Median_Direction_Acc','Mean_RMSE','Median_RMSE'])
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[0], 'Gamma(Uniform_Exponential)')
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[1], 'Gamma(Uniform_Uniform)')
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[2], 'Gamma(Exponential_Exponential)')
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[3], 'LogNormal(Uniform_Exponential)')
#HV_Results.head()
styled = HV_Results.style.background_gradient(axis = 1)
styled.set_table_attributes("style='display:inline'").set_caption('All Tickers')
styled

,Model,Credible_Interval,Mean_Median_Interval,Mean_Direction_Acc,Median_Direction_Acc,Mean_RMSE,Median_RMSE
0,Gamma(Uniform_Exponential),0.938080,0.841073,0.862745,0.668731,0.293413,0.321763
1,Gamma(Uniform_Uniform),0.983488,0.963880,0.758514,0.829721,0.471844,0.315584
2,Gamma(Exponential_Exponential),0.912281,0.514964,0.863777,0.834881,0.293336,0.309542
3,LogNormal(Uniform_Exponential),0.865841,0.934985,0.725490,0.756450,7.770775,0.800073


In [25]:
HV_Results = pd.DataFrame(columns = ['Model','Credible_Interval','Mean_Median_Interval','Mean_Direction_Acc','Median_Direction_Acc','Mean_RMSE','Median_RMSE'])
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[0], 'Gamma(Uniform_Exponential)',sectors = High_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[1], 'Gamma(Uniform_Uniform)',sectors = High_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[2], 'Gamma(Exponential_Exponential)',sectors = High_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[3], 'LogNormal(Uniform_Exponential)',sectors = High_vol_sectors)

#HV_Results.head()
styled = HV_Results.style.background_gradient(axis = 1)
styled.set_table_attributes("style='display:inline'").set_caption('High Volatility Sectors')
styled

,Model,Credible_Interval,Mean_Median_Interval,Mean_Direction_Acc,Median_Direction_Acc,Mean_RMSE,Median_RMSE
0,Gamma(Uniform_Exponential),0.948718,0.839744,0.833333,0.675214,0.257950,0.281823
1,Gamma(Uniform_Uniform),0.989316,0.974359,0.745726,0.807692,0.467212,0.281718
2,Gamma(Exponential_Exponential),0.910256,0.487179,0.833333,0.818376,0.258006,0.268151
3,LogNormal(Uniform_Exponential),0.835470,0.901709,0.735043,0.745726,6.687688,0.780439


In [26]:
HV_Results = pd.DataFrame(columns = ['Model','Credible_Interval','Mean_Median_Interval','Mean_Direction_Acc','Median_Direction_Acc','Mean_RMSE','Median_RMSE'])
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[0], 'Gamma(Uniform_Exponential)',sectors = Medium_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[1], 'Gamma(Uniform_Uniform)',sectors = Medium_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[2], 'Gamma(Exponential_Exponential)',sectors = Medium_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[3], 'LogNormal(Uniform_Exponential)',sectors = Medium_vol_sectors)

#HV_Results.head()
styled = HV_Results.style.background_gradient(axis = 1)
styled.set_table_attributes("style='display:inline'").set_caption('Medium Volatility Sectors')
styled

,Model,Credible_Interval,Mean_Median_Interval,Mean_Direction_Acc,Median_Direction_Acc,Mean_RMSE,Median_RMSE
0,Gamma(Uniform_Exponential),0.929878,0.838415,0.902439,0.661585,0.334487,0.368636
1,Gamma(Uniform_Uniform),0.978659,0.948171,0.783537,0.871951,0.484183,0.354126
2,Gamma(Exponential_Exponential),0.914634,0.545732,0.905488,0.856707,0.334384,0.354913
3,LogNormal(Uniform_Exponential),0.884146,0.957317,0.722561,0.777439,6.676078,0.804130


In [27]:
HV_Results = pd.DataFrame(columns = ['Model','Credible_Interval','Mean_Median_Interval','Mean_Direction_Acc','Median_Direction_Acc','Mean_RMSE','Median_RMSE'])
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[0], 'Gamma(Uniform_Exponential)',sectors = Low_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[1], 'Gamma(Uniform_Uniform)',sectors = Low_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[2], 'Gamma(Exponential_Exponential)',sectors = Low_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_HV(data)[3], 'LogNormal(Uniform_Exponential)',sectors = Low_vol_sectors)

#HV_Results.head()
styled = HV_Results.style.background_gradient(axis = 1)
styled.set_table_attributes("style='display:inline'").set_caption('Low Volatility Sectors')
styled

,Model,Credible_Interval,Mean_Median_Interval,Mean_Direction_Acc,Median_Direction_Acc,Mean_RMSE,Median_RMSE
0,Gamma(Uniform_Exponential),0.924855,0.849711,0.867052,0.664740,0.300148,0.327710
1,Gamma(Uniform_Uniform),0.976879,0.965318,0.745665,0.809249,0.460475,0.324614
2,Gamma(Exponential_Exponential),0.913295,0.531792,0.867052,0.838150,0.299816,0.321470
3,LogNormal(Uniform_Exponential),0.913295,0.982659,0.705202,0.745665,11.520943,0.843639


# The next four cells are for the IV predictions

In [28]:
HV_Results = pd.DataFrame(columns = ['Model','Credible_Interval','Mean_Median_Interval','Mean_Direction_Acc','Median_Direction_Acc','Mean_RMSE','Median_RMSE'])
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[0], 'Gamma(Uniform_Exponential)')
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[1], 'Gamma(Uniform_Uniform)')
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[2], 'Gamma(Exponential_Exponential)')
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[3], 'LogNormal(Uniform_Exponential)')


#HV_Results.head()
styled = HV_Results.style.background_gradient(axis = 1)
styled.set_table_attributes("style='display:inline'").set_caption('All Tickers')
styled

,Model,Credible_Interval,Mean_Median_Interval,Mean_Direction_Acc,Median_Direction_Acc,Mean_RMSE,Median_RMSE
0,Gamma(Uniform_Exponential),0.913313,0.808050,0.466460,0.665635,0.286908,0.326675
1,Gamma(Uniform_Uniform),0.970072,0.921569,0.299278,0.351909,0.434633,0.300130
2,Gamma(Exponential_Exponential),0.787410,0.269350,0.464396,0.540764,0.286869,0.306638
3,LogNormal(Uniform_Exponential),0.538700,0.926729,0.318885,0.308566,7.739948,0.757874


In [29]:
HV_Results = pd.DataFrame(columns = ['Model','Credible_Interval','Mean_Median_Interval','Mean_Direction_Acc','Median_Direction_Acc','Mean_RMSE','Median_RMSE'])
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[0], 'Gamma(Uniform_Exponential)',sectors = High_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[1], 'Gamma(Uniform_Uniform)',sectors = High_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[2], 'Gamma(Exponential_Exponential)',sectors = High_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[3], 'LogNormal(Uniform_Exponential)',sectors = High_vol_sectors)

#HV_Results.head()
styled = HV_Results.style.background_gradient(axis = 1)
styled.set_table_attributes("style='display:inline'").set_caption('High Volatility Sectors')
styled

,Model,Credible_Interval,Mean_Median_Interval,Mean_Direction_Acc,Median_Direction_Acc,Mean_RMSE,Median_RMSE
0,Gamma(Uniform_Exponential),0.923077,0.809829,0.487179,0.653846,0.294327,0.332763
1,Gamma(Uniform_Uniform),0.985043,0.931624,0.324786,0.399573,0.448290,0.307149
2,Gamma(Exponential_Exponential),0.779915,0.269231,0.489316,0.542735,0.294447,0.310374
3,LogNormal(Uniform_Exponential),0.534188,0.888889,0.337607,0.329060,6.651224,0.744328


In [30]:
HV_Results = pd.DataFrame(columns = ['Model','Credible_Interval','Mean_Median_Interval','Mean_Direction_Acc','Median_Direction_Acc','Mean_RMSE','Median_RMSE'])
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[0], 'Gamma(Uniform_Exponential)',sectors = Medium_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[1], 'Gamma(Uniform_Uniform)',sectors = Medium_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[2], 'Gamma(Exponential_Exponential)',sectors = Medium_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[3], 'LogNormal(Uniform_Exponential)',sectors = Medium_vol_sectors)

#HV_Results.head()
styled = HV_Results.style.background_gradient(axis = 1)
styled.set_table_attributes("style='display:inline'").set_caption('Medium Volatility Sectors')
styled

,Model,Credible_Interval,Mean_Median_Interval,Mean_Direction_Acc,Median_Direction_Acc,Mean_RMSE,Median_RMSE
0,Gamma(Uniform_Exponential),0.896341,0.798780,0.454268,0.667683,0.307367,0.350636
1,Gamma(Uniform_Uniform),0.960366,0.902439,0.289634,0.320122,0.437918,0.319673
2,Gamma(Exponential_Exponential),0.789634,0.265244,0.442073,0.539634,0.307214,0.330943
3,LogNormal(Uniform_Exponential),0.548780,0.948171,0.307927,0.298780,6.648797,0.758297


In [31]:
HV_Results = pd.DataFrame(columns = ['Model','Credible_Interval','Mean_Median_Interval','Mean_Direction_Acc','Median_Direction_Acc','Mean_RMSE','Median_RMSE'])
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[0], 'Gamma(Uniform_Exponential)',sectors = Low_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[1], 'Gamma(Uniform_Uniform)',sectors = Low_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[2], 'Gamma(Exponential_Exponential)',sectors = Low_vol_sectors)
HV_Results.loc[len(HV_Results.index)] = condense_results(split_dataframes_IV(data)[3], 'LogNormal(Uniform_Exponential)',sectors = Low_vol_sectors)

#HV_Results.head()
styled = HV_Results.style.background_gradient(axis = 1)
styled.set_table_attributes("style='display:inline'").set_caption('Low Volatility Sectors')
styled

,Model,Credible_Interval,Mean_Median_Interval,Mean_Direction_Acc,Median_Direction_Acc,Mean_RMSE,Median_RMSE
0,Gamma(Uniform_Exponential),0.919075,0.820809,0.433526,0.693642,0.218175,0.255122
1,Gamma(Uniform_Uniform),0.947977,0.930636,0.248555,0.283237,0.388397,0.235759
2,Gamma(Exponential_Exponential),0.803468,0.277457,0.439306,0.537572,0.217859,0.241681
3,LogNormal(Uniform_Exponential),0.531792,0.988439,0.289017,0.271676,11.491690,0.792597
